In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("F:\PGDDS\PGD_Data_Science\csv_data\AML.csv")

In [4]:
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [5]:
df.shape

(6362620, 11)

In [6]:
df.isnull().sum()

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [9]:
l = ["type","isFraud","isFlaggedFraud"]
for i in l:
    print(i,"-",df[i].unique())

type - ['PAYMENT' 'TRANSFER' 'CASH_OUT' 'DEBIT' 'CASH_IN']
isFraud - [0 1]
isFlaggedFraud - [0 1]


In [10]:
df1 = df.drop(["step","nameOrig", "nameDest"], axis = 1)

In [11]:
df1.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,PAYMENT,9839.64,170136.0,160296.36,0.0,0.0,0,0
1,PAYMENT,1864.28,21249.0,19384.72,0.0,0.0,0,0
2,TRANSFER,181.00,181.0,0.00,0.0,0.0,1,0
3,CASH_OUT,181.00,181.0,0.00,21182.0,0.0,1,0
4,PAYMENT,11668.14,41554.0,29885.86,0.0,0.0,0,0


In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 8 columns):
 #   Column          Dtype  
---  ------          -----  
 0   type            object 
 1   amount          float64
 2   oldbalanceOrg   float64
 3   newbalanceOrig  float64
 4   oldbalanceDest  float64
 5   newbalanceDest  float64
 6   isFraud         int64  
 7   isFlaggedFraud  int64  
dtypes: float64(5), int64(2), object(1)
memory usage: 388.3+ MB


In [13]:
from sklearn.preprocessing import MinMaxScaler

In [14]:
scaler = MinMaxScaler()

In [15]:
for i in df1:
    if i != "type":
        df1[i] = scaler.fit_transform(df1[[i]])

In [16]:
df2 = pd.get_dummies(data = df1 , columns = ["type"], drop_first = True)
df2.head()

,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud,type_CASH_OUT,type_DEBIT,type_PAYMENT,type_TRANSFER
0,0.000106,0.002855,0.003233,0.000000,0.0,0.0,0.0,0,0,1,0
1,0.000020,0.000357,0.000391,0.000000,0.0,0.0,0.0,0,0,1,0
2,0.000002,0.000003,0.000000,0.000000,0.0,1.0,0.0,0,0,0,1
3,0.000002,0.000003,0.000000,0.000059,0.0,1.0,0.0,1,0,0,0
4,0.000126,0.000697,0.000603,0.000000,0.0,0.0,0.0,0,0,1,0


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   amount          float64
 1   oldbalanceOrg   float64
 2   newbalanceOrig  float64
 3   oldbalanceDest  float64
 4   newbalanceDest  float64
 5   isFraud         float64
 6   isFlaggedFraud  float64
 7   type_CASH_OUT   uint8  
 8   type_DEBIT      uint8  
 9   type_PAYMENT    uint8  
 10  type_TRANSFER   uint8  
dtypes: float64(7), uint8(4)
memory usage: 364.1 MB


In [18]:
X = df2.drop("isFlaggedFraud", axis = 1)
y = df2["isFlaggedFraud"]

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [21]:
from sklearn.linear_model import LogisticRegression

In [22]:
Lr = LogisticRegression()
Lr.fit(X_train, y_train)
Lr.score(X_test, y_test)

0.9999976424806133

In [23]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.9999984283204089